In [1]:
import time
from datetime import datetime
import argparse
import json
import threading
import queue
import os

import cv2
import numpy as np
import torch
import redis
import matplotlib.pyplot as plt

from core import support, share_param
from insight_face.modules.tracking.custom_tracking import TrackingMultiCam
from insight_face.utils.database import FaceRecognitionSystem

In [2]:
share_param.sdk_config = support.get_config_yaml("configs/sdk_config.yaml")
share_param.dev_config = support.get_config_yaml("configs/dev_config.yaml")
share_param.facerec_system = FaceRecognitionSystem(share_param.dev_config["DATA"]["photo_path"], share_param.sdk_config )

IResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=2e-05, momentum=0.9, affine=True, track_running_stats=True)
  (prelu): PReLU(num_parameters=64)
  (layer1): Sequential(
    (0): IBasicBlock(
      (bn1): BatchNorm2d(64, eps=2e-05, momentum=0.9, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=2e-05, momentum=0.9, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=64)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=2e-05, momentum=0.9, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (1): BatchNorm2d(64, eps=2e-05, momentum=0.9, affine=True, track_running_stats=True)
      )
    )
    (1): IBas

Using cache found in /home/khiemtv/.cache/torch/hub/ultralytics_yolov5_master
Fusing layers... 
Model Summary: 224 layers, 7059304 parameters, 0 gradients, 16.3 GFLOPs
Adding AutoShape... 
YOLOv5 🚀 2021-6-16 torch 1.8.1 CUDA:0 (GeForce RTX 2060, 5931.4375MB)

Finish SDK initialization


['mask', 'glasses', 'hand']


In [6]:
#CROP
count = 0
for root, dirs, files in os.walk("/mnt/LINUXDATA/Source/.data/face_attributes/mask02", topdown=False):
    # print(root)
    for name in files:
        count+=1
        path = os.path.join(root, name)
        bgr = cv2.imread(path)
        rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
        bboxes, landmarks = share_param.facerec_system.sdk.detect_faces(rgb)
        # print(bboxes)
        # print(landmarks)
        for bbox, landmark in zip(bboxes, landmarks):
            faceW = bbox[2] - bbox[0]
            faceH = bbox[3] - bbox[1]
            expandLeft = max(0, bbox[0] - faceW/3)
            expandTop = max(0, bbox[1] - faceH/3)
            expandRight = min(bbox[2] + faceW/3, bgr.shape[1])
            expandBottom = min(bbox[3] + faceH/3, bgr.shape[0])
            faceCropExpand = bgr[int(expandTop):int(expandBottom), int(expandLeft):int(expandRight)].copy()
            cv2.imwrite(f"dataset/mask02/{count:04d}.jpg", faceCropExpand)


KeyboardInterrupt: 

In [ ]:
def check_not_blur(image: np.ndarray):
    if image is None or image.size == 0:
        return False, 0.0

    # gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    imageresize = cv2.resize(image, (112,112))
    real_notblur = cv2.Laplacian(imageresize, cv2.CV_64F).var()
    standard_notblur = 300
    threshnotblur = real_notblur/standard_notblur
    print(threshnotblur)

    if threshnotblur < 1.5:
        return False, threshnotblur
    else:
        return True, threshnotblur
    return False, threshnotblur

In [ ]:
def illumination(image):
    image = cv2.resize(image, (112,112))
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # length of R available  range  of  gray  intensities  excluding  5%  of  the darkest  and  brightest  pixel
    sorted_gray = np.sort(gray.ravel())
    l = len(sorted_gray)
    cut_off_idx = l * 5 // 100
    r = sorted_gray[l-cut_off_idx] - sorted_gray[cut_off_idx]
    return np.round(r / 255, 2)

### Check quality
Blur and Straight

In [ ]:

count = 0
for root, dirs, files in os.walk("dataset/bestphotos", topdown=False):
    # print(root)
    for name in files:
        count+=1
        if count>100:
            break
        # print(name)
        # if name != "focus_02550.jpg":
        #     continue
        path = os.path.join(root, name)
        rgb = cv2.imread(path)
        # rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
        bboxes, landmarks = share_param.facerec_system.sdk.detect_faces(rgb)
        # print(bboxes)
        # print(landmarks)
        for bbox, landmark in zip(bboxes, landmarks):
            faceSize = float((bbox[3]-bbox[1])*(bbox[2]-bbox[0]))
            faceAlign = share_param.facerec_system.sdk.align_face(rgb, landmark)
            faceCrop = rgb[int(bbox[1]):int(bbox[3]),
                                    int(bbox[0]):int(bbox[2])]

            if faceCrop.shape[0] < 50 or faceCrop.shape[1] < 50:
                continue
            isillumination, threshillumination = share_param.facerec_system.sdk.evaluter.check_illumination(faceCrop)
            isNotBlur, threshnotblur = share_param.facerec_system.sdk.evaluter.check_not_blur(faceCrop)
            isStraightFace = share_param.facerec_system.sdk.evaluter.check_straight_face(rgb, landmark)

            # isNotBlur, threshnotblur = share_param.facerec_system.sdk.evaluter.check_not_blur(faceCrop)
            # illuminate = share_param.facerec_system.sdk.evaluter.check_illumination(faceCrop)
            # isNotBlur, threshnotblur = share_param.facerec_system.sdk.evaluter.detect_blur_fft(faceAlign, size=30)
            # isStraightFace = share_param.facerec_system.sdk.evaluter.check_straight_face(rgb, landmark)
            # print("Not blur:", isNotBlur)
            # print("Straight:", isStraightFace)
            cnts = landmark.reshape(5,2, order='F')
            # for point in cnts:
            #     cv2.drawMarker(rgb, tuple(point), color=(0,255,0), markerType=cv2.MARKER_CROSS, markerSize=8, thickness=1)
            plt.figure()
            plt.title(f"{name} {threshillumination} {threshnotblur} {isStraightFace}")
            plt.imshow(faceCrop)

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

img = cv2.imread('datalab/images/20210507145554353919.jpg')
# img = cv2.resize(img, (224,224))
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


In [ ]:

minsize = 20 # minimum size of face
threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
factor = 0.709 # scale factor

#Size Parameter
lower_threshold = 100
upper_threshold = 200

bboxes, points = share_param.facerec_system.sdk.detect_faces(img)

print(bboxes, points)


In [ ]:
def illumination(gray):
    # length of R available  range  of  gray  intensities  excluding  5%  of  the darkest  and  brightest  pixel
    sorted_gray = np.sort(gray.ravel())
    l = len(sorted_gray)
    cut_off_idx = l * 5 // 100
    r = sorted_gray[l-cut_off_idx] - sorted_gray[cut_off_idx]
    return np.round(r / 255, 2)


In [ ]:
illumination(gray)

In [ ]:
def get_contour(pts):
    return np.array([[pts[i], pts[5 + i]] for i in  [0, 1, 4, 3]], np.int32).reshape((-1,1,2))
def get_mask(image, contour):
    mask = np.zeros(image.shape[0:2],dtype="uint8")
    cv2.drawContours(mask, [contour], -1, 255, -1)
    return mask

In [ ]:
def sharpness(img, landmark):
    contour = get_contour(landmark)
    mask = get_mask(img, contour) #1-channel mask
    # plt.imshow(mask)
    mask = np.stack((mask,)*3, axis=-1) #3-channel mask
    mask[mask == 255] = 1 # convert 0 and 255 to 0 and 1
    laplacian = cv2.Laplacian(img,cv2.CV_64F)
    # print(laplacian)
    # print(laplacian.var())
    edges = laplacian[mask.astype(bool)]
    return np.round(edges.var() / 255 , 2)

plt.imshow(img)
sharpness(img, points[0])

In [ ]:
from skimage.feature import hog

def symmetry(img, landmark, bounding_box):
    x1, y1, x2, y2 = int(min(bounding_box[0], min(landmark[:5]))), \
        int(min(bounding_box[1], min(landmark[5:]))), \
        int(max(bounding_box[2], max(landmark[:5]))), \
        int(max(bounding_box[3], max(landmark[5:])))
    
    landmark = np.array([[landmark[i], landmark[5 + i]] for i in  range(5)], np.int32).reshape((-1,1,2))
    contour = landmark[:, 0] - [[x1, y1]]
    
    face = img[y1: y2, x1: x2].copy()
    face_flip = cv2.flip(face, 1)
    
    fd_face, hog_face = hog(face, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)
    fd_flip, hog_flip = hog(face_flip, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)
    
    
    d = np.zeros(len(contour))
    for i in range(len(d)):
        d[i] = min(hog_face[contour[i, 1], contour[i, 0]], hog_flip[contour[i, 1], contour[i, 0]])
    
    return np.average(d)

In [ ]:
symmetry(img, points[0], bboxes[0])

In [ ]:
plt.figure()
for root, dirs, files in os.walk("datalab/images/", topdown=False):
    # print(root)
    for name in files:
        path = os.path.join(root, name)
        img = cv2.imread(path)
        # img = cv2.resize(img, (224,224))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        bboxes, points = share_param.facerec_system.sdk.detect_faces(img)
        if len(bboxes) == 0:
            continue
        # print(bboxes)
        # print(points)
        # print("illumination",illumination(gray))
        # print("sharpness",sharpness(img, points[0]))
        # print("symmetry", symmetry(img, points[0], bboxes[0]))

        cv2.rectangle(img, (int(bboxes[0][0]), int(bboxes[0][1])), (int(bboxes[0][2]), int(bboxes[0][3])), (0, 255, 0), 2)
        plt.figure()
        strInfo = f"{illumination(gray)} {sharpness(img, points[0])} {symmetry(img, points[0], bboxes[0])}"
        plt.title(strInfo)
        plt.imshow(img)



In [ ]:
sharpness 0.27
symmetry 0.11464042663574218

### TEST CARD ID

In [ ]:
import time
from datetime import datetime
import argparse
import json
import threading
import queue
import os

import cv2
import numpy as np
import torch
import redis
import matplotlib.pyplot as plt

# from core import support, share_param
# from insight_face.modules.tracking.custom_tracking import TrackingMultiCam
# from insight_face.utils.database import FaceRecognitionSystem

In [ ]:
def prepare(image):
    h= image.shape[0]
    w= image.shape[1]

    minsize = int(min(w,h)*2/3)

    # print(w,h)
    # print(minsize)

    xstart = (w - minsize)//2
    ystart = (h - minsize)//2
    frame = image[ystart:ystart+minsize, xstart: xstart + minsize]
    return frame

def check_illumination(self, image):
    if image is None or image.size == 0:
        return False, 0.0
    image = cv2.resize(image, (112,112))
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # length of R available  range  of  gray  intensities  excluding  5%  of  the darkest  and  brightest  pixel
    sorted_gray = np.sort(gray.ravel())
    l = len(sorted_gray)
    cut_off_idx = l * 5 // 100
    r = sorted_gray[l-cut_off_idx] - sorted_gray[cut_off_idx]
    illuminate = np.round(r / 255, 2)

    if illuminate < self.illumination_threshold:
        return False, illuminate

    return True, illuminate

def check_illuminationhsv(self, image):
    if image is None or image.size == 0:
        return False, 0.0
    image = cv2.resize(image, (112,112))
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    v = hsv[:,:,2]


    plt.figure()
    plt.hist(v.ravel(),256,[0,256])
    plt.show()

    # sorted_gray = np.sort(v.ravel())
    # l = len(sorted_gray)
    # cut_off_idx = l * 5 // 100
    # r = sorted_gray[l-cut_off_idx] - sorted_gray[cut_off_idx]
    # illuminate = np.round(r / 255, 2)

    illuminate = v.std()

    if illuminate < self.illumination_threshold:
        return False, illuminate

    return True, illuminate   
    # print(v)
    # plt.figure()
    # plt.hist(v.ravel(), bins=256, range=(0.0, 1.0), fc='k', ec='k') #calculating histogram

    # print(v.shape)

    # plt.figure()
    # plt.title("brightvalue")
    # plt.imshow(v, cmap='gray')

    return True , 10



    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # length of R available  range  of  gray  intensities  excluding  5%  of  the darkest  and  brightest  pixel
    sorted_gray = np.sort(gray.ravel())
    l = len(sorted_gray)
    cut_off_idx = l * 5 // 100
    r = sorted_gray[l-cut_off_idx] - sorted_gray[cut_off_idx]
    illuminate = np.round(r / 255, 2)

    if illuminate < self.illumination_threshold:
        return False, illuminate

    return True, illuminate        

def check_not_blur(self, image: np.ndarray):
    if image is None or image.size == 0:
        return False, 0.0

    image = cv2.resize(image, (112,112))

    real_notblur = cv2.Laplacian(image, cv2.CV_64F).var()
    standard_notblur = 300

    threshnotblur = real_notblur/standard_notblur
    # print(threshnotblur)

    if threshnotblur < self.blur_threshold:
        return False, threshnotblur
    else:
        return True, threshnotblur
    return False, threshnotblur

In [ ]:
count = 0

class Param:
    illumination_threshold = 0.8
    blur_threshold = 0.8

for root, dirs, files in os.walk("/mnt/LINUXDATA/Source/data/datacmt", topdown=False):
    if count>100:
            break
    for name in files:
        count+=1
        
        path = os.path.join(root, name)
        bgr = cv2.imread(path)
        bgr = prepare(bgr)
        # print(bgr.shape)
        rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
        isNotBlur, threshnotblur = check_not_blur(Param, rgb)
        illuminate, threshillumination = check_illuminationhsv(Param, rgb)
        strR = "{:03.3f} {:03.3f}".format(threshnotblur, threshillumination)
        pt1 = [10, 20]
        fontFace = cv2.FONT_HERSHEY_COMPLEX_SMALL
        fontScale = 0.5
        thickness = 1
        retval, baseLine = cv2.getTextSize(strR,fontFace=fontFace,fontScale=fontScale, thickness=thickness)
        # Calculate the coordinates of the rectangular frame covering the text
        topleft = (pt1[0], pt1[1] - retval[1])
        bottomright = (topleft[0] + retval[0], topleft[1] + retval[1])
        cv2.rectangle(rgb, (topleft[0], topleft[1] - baseLine), bottomright,thickness=-1, color=(0, 255, 0))
        # Draw text
        cv2.putText(rgb, strR, (pt1[0], pt1[1]-baseLine), fontScale=fontScale,fontFace=fontFace, thickness=thickness, color=(0,0,0))

        # cv2.rectangle(bgr, (10, 10-textSize[1]), (10 + int(textSize[0][0]), 10 + int(textSize[0][1])), (0,0,0), -1)
    
        # cv2.putText(bgr, "{:03.3f} {:03.3f}".format(threshnotblur, threshillumination), (10,10-textSize[1]), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.5, (0,255,0), 1)
        plt.figure()
        plt.title(strR)
        plt.imshow(rgb)
        savename = "R"+name
        cv2.imwrite(os.path.join("/mnt/LINUXDATA/Source/data/", savename),cv2.cvtColor(rgb,cv2.COLOR_RGB2BGR))

#### CONVERT JIT

In [ ]:
import time
from datetime import datetime
import argparse
import json
import threading
import queue
import os
import glob

import cv2
import numpy as np
import torch
import redis
import matplotlib.pyplot as plt
from core import support

from insight_face.modules.detection.retinaface.model_class import RetinaFace

# from core import support, share_param
# from insight_face.modules.tracking.custom_tracking import TrackingMultiCam
# from insight_face.utils.database import FaceRecognitionSystem

In [ ]:
sdk_config = support.get_config_yaml("configs/sdk_config.yaml")
# share_param.dev_config = support.get_config_yaml("configs/dev_config.yaml")
# share_param.facerec_system = FaceRecognitionSystem(share_param.dev_config["DATA"]["photo_path"], share_param.sdk_config )

detector = RetinaFace(sdk_config["detector"])

In [ ]:
bgr = cv2.imread("dataset/bestphotos/113900549523G.jpg")
bgr = cv2.resize(bgr, (336,336))
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
# preTime = time.time()
image = detector._preprocess(rgb)
detector.model_input_shape = image.shape

img = image.transpose((2, 0, 1))
img = torch.from_numpy(img).unsqueeze(0)
img = img.to(detector.device)
# pred = detector.model(img)

detector.model = torch.jit.trace(detector.model,img)
# print(pred)

# raw_pred = detector._predict_raw(image)

# bgr = cv2.resize(bgr, (336,336))
# bgr = cv2.resize(bgr, (336,336))
# rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
# detector.predict(image)
# traced_predict = torch.jit.trace(detector._predict_raw, image)

In [ ]:
detector.model.half()
for filename in glob.glob("/mnt/LINUXDATA/Source/.data/face_attributes/01000-20210525T030241Z-001/01000/*.png"):
    bgr = cv2.imread(filename)
    bgr = cv2.resize(bgr, (336,336))
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    image = detector._preprocess(rgb)
    detector.model_input_shape = image.shape
    preTime = time.time()
    # image.astype(np.float16)
    raw_pred = detector._predict_raw(image)
    print("Detect:", time.time()-preTime)

    bboxes, landms = detector._postprocess(raw_pred)


#### EXPORT OPENVINO

##### EXPORT RETINA

In [ ]:
import torch
from insight_face.modules.detection.retinaface.model_class import RetinaFace
import cv2
from core import support

config = {
    "image_size": 336,                   # image size input to detector. Default=224
    "nms_threshold": 0.5,                   # nms
    "conf_threshold": 0.8,               # confidence
    "minface": 50,                       # min face size. Default=50
    "device": "cpu",                      # cpu, cuda
    "architecture": "mnet1"               # res50 (ResNet 50), mnet1 (mobilenet1)
}

detector = RetinaFace(config)

dummy_input = torch.zeros((1,3,config["image_size"],config["image_size"]))
torch.onnx.export(detector.model, dummy_input, f'weights/{sdk_config["detector"]["architecture"]}.onnx', opset_version=11)

! python ~/intel/openvino_2021.3.394/deployment_tools/model_optimizer/mo_onnx.py --input_model=weights/mnet1.onnx --output_dir=weights 
# --data_type=FP16

In [ ]:
import torch
from insight_face.modules.detection.retinaface.model_class import RetinaFace
import cv2
from core import support

config = {
    "image_size": 336,                   # image size input to detector. Default=224
    "nms_threshold": 0.5,                   # nms
    "conf_threshold": 0.8,               # confidence
    "minface": 50,                       # min face size. Default=50
    "device": "cpu",                      # cpu, cuda
    "architecture": "res50"               # res50 (ResNet 50), mnet1 (mobilenet1)
}

detector = RetinaFace(config)

dummy_input = torch.zeros((1,3,config["image_size"],config["image_size"]))
torch.onnx.export(detector.model, dummy_input, f'weights/{config["architecture"]}.onnx', opset_version=11)

! python ~/intel/openvino_2021.3.394/deployment_tools/model_optimizer/mo_onnx.py --input_model=weights/res50.onnx --output_dir=weights 
# --data_type=FP16

##### EXPORT FACE EMMBED

In [ ]:
import torch
from insight_face.modules.recognition.insightface.insightface import InsightFaceEmbedder
import cv2
from core import support

config = {
    "image_size": 112,                   # input size of emmbeder 
    "descriptor_size": 512,
    "device": "cpu",                      # cpu, cuda
    "architecture": "iresnet34"           # iresnet100, iresnet50, iresnet34
}

face_embedder = InsightFaceEmbedder(config)

dummy_input = torch.zeros((1,3,config["image_size"],config["image_size"]))
torch.onnx.export(face_embedder.embedder, dummy_input, f'weights/{config["architecture"]}.onnx', opset_version=11)

! python ~/anaconda3/envs/openvino/lib/python3.7/site-packages/mo_onnx.py --input_model=weights/iresnet34.onnx --output_dir=weights 

In [ ]:
import torch
from insight_face.modules.recognition.insightface.insightface import InsightFaceEmbedder
import cv2
from core import support

config = {
    "image_size": 112,                   # input size of emmbeder 
    "descriptor_size": 512,
    "device": "cpu",                      # cpu, cuda
    "architecture": "iresnet50"           # iresnet100, iresnet50, iresnet34
}

face_embedder = InsightFaceEmbedder(config)

dummy_input = torch.zeros((1,3,config["image_size"],config["image_size"]))
torch.onnx.export(face_embedder.embedder, dummy_input, f'weights/{config["architecture"]}.onnx', opset_version=11)

! python ~/anaconda3/envs/openvino/lib/python3.7/site-packages/mo_onnx.py --input_model=weights/iresnet50.onnx --output_dir=weights 

In [ ]:
import torch
from insight_face.modules.recognition.insightface.insightface import InsightFaceEmbedder
import cv2
from core import support

config = {
    "image_size": 112,                   # input size of emmbeder 
    "descriptor_size": 512,
    "device": "cpu",                      # cpu, cuda
    "architecture": "iresnet100"           # iresnet100, iresnet50, iresnet34
}

face_embedder = InsightFaceEmbedder(config)

dummy_input = torch.zeros((1,3,config["image_size"],config["image_size"]))
torch.onnx.export(face_embedder.embedder, dummy_input, f'weights/{config["architecture"]}.onnx', opset_version=11)

! python ~/anaconda3/envs/openvino/lib/python3.7/site-packages/mo_onnx.py --input_model=weights/iresnet100.onnx --output_dir=weights 

In [ ]:
import os
import numpy as np
# os.environ['LD_LIBRARY_PATH'] = "/home/khiemtv/anaconda3/envs/openvino/lib"
# print(os.environ['LD_LIBRARY_PATH'])
from openvino.inference_engine import IECore

ie = IECore()
net = ie.read_network("weights/iresnet34.xml", "weights/iresnet34.bin")
exec_net = ie.load_network(network=net, device_name="CPU")
image = np.zeros((1,3,112,112))
# data[input_name] = images
# print(face_embedder.embedder(dummy_input))

ten_res = face_embedder.embedder(dummy_input).detach().numpy()
res = exec_net.infer(inputs={"input.1": image})
# print(res)
# print(abs((ten_res - res['475'])/.sum())/)
